<a href="https://colab.research.google.com/github/Arpit1118/Post-Training-LLMs-with-RL/blob/main/LLM_using_RL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Core libraries
import torch
import random
import numpy as np

# Transformers from Hugging Face
from transformers import AutoTokenizer, AutoModelForCausalLM

# Evaluation and logging
import time
import traceback
import logging

# Set seeds for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

# Using Qwen2.5 model
MODEL_NAME = "Qwen/Qwen2.5-0.5B"

# Load tokenizer and model
tokenizer =  AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)
device = torch.device("cpu")
model.to(device)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((896,), eps=1e-06)
    (rotary_emb): Qwen2RotaryEmbe

In [2]:
import sympy as sp

class MathSolver:
    def __init__(self, variable='x'):
        self.x = sp.Symbol(variable)

    def solve_equation(self, equation_str):
        """
        Solves an equation like 'x**2 - 4 = 0' or 'sin(x) = 0'
        Returns symbolic and numeric solutions
        """
        try:
            if '=' in equation_str:
                lhs, rhs = equation_str.split('=')
                expr = sp.sympify(lhs) - sp.sympify(rhs)
            else:
                expr = sp.sympify(equation_str)

            roots = sp.solve(expr, self.x)
            numeric = [sp.N(r) for r in roots]

            return {
                "success": True,
                "symbolic": roots,
                "numeric": numeric,
                "error": None
            }
        except Exception as e:
            return {
                "success": False,
                "symbolic": None,
                "numeric": None,
                "error": str(e)
            }

    def evaluate_expression(self, expr_str):
        """
        Evaluates a basic math expression like '2 + 3 * 4'
        """
        try:
            result = sp.sympify(expr_str).evalf()
            return {
                "success": True,
                "result": result,
                "error": None
            }
        except Exception as e:
            return {
                "success": False,
                "result": None,
                "error": str(e)
            }

if __name__ == "__main__":
    solver = MathSolver()
    print("Math Solver Ready. Type 'exit' to quit.")
    while True:
        inp = input(">>> ")
        if inp.lower() in ['exit', 'quit']:
            break
        if '=' in inp:
            out = solver.solve_equation(inp)
            if out["success"]:
                print(f"Symbolic: {out['symbolic']}")
                print(f"Numeric: {out['numeric']}")
            else:
                print("Error:", out["error"])
        else:
            out = solver.evaluate_expression(inp)
            if out["success"]:
                print("Result:", out["result"])
            else:
                print("Error:", out["error"])

Math Solver Ready. Type 'exit' to quit.
>>> x**3 - 6*x**2 + 11*x - 6 = 0
Symbolic: [1, 2, 3]
Numeric: [1.00000000000000, 2.00000000000000, 3.00000000000000]
>>> sqrt(x + 5) = x - 1
Symbolic: [4]
Numeric: [4.00000000000000]
>>> tan(x) - sqrt(3) = 0
Symbolic: [pi/3]
Numeric: [1.04719755119660]
>>> sin(x)**2 + cos(x)**2 - 1 = 0
Symbolic: []
Numeric: []
>>> sqrt(2)**2 + log(exp(1))
Result: 3.00000000000000
>>> quit


In [3]:
import re
from functools import lru_cache

class LLMMathWrapper:
    def __init__(self, model, tokenizer, math_solver):
        self.model = model
        self.tokenizer = tokenizer
        self.math_solver = math_solver

    def is_math_prompt(self, prompt):
        prompt = prompt.strip()

        # Removed punctuation for better word analysis
        cleaned = re.sub(r'[^\w\s]', '', prompt.lower())

        # Quick pass for math symbols or math keywords
        if re.search(r'[\d\+\-\*/\^=()]', prompt):
            math_keywords = {"solve", "evaluate", "simplify", "integrate", "differentiate", "factor"}
            if any(kw in cleaned for kw in math_keywords):
                return True

            words = cleaned.split()
            if len(words) <= 5 or sum(w.isdigit() or w in {'x', 'y'} for w in words) >= len(words) // 2:
                return True

        return False

    @lru_cache(maxsize=128)
    def generate_with_llm(self, prompt):
        inputs = self.tokenizer.encode(prompt, return_tensors="pt")
        outputs = self.model.generate(inputs, max_length=100, num_return_sequences=1)
        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)

    def run(self, prompt):
        if self.is_math_prompt(prompt):
            if '=' in prompt:
                result = self.math_solver.solve_equation(prompt)
                if result["success"]:
                    return f"Symbolic: {result['symbolic']}\nNumeric: {result['numeric']}"
                else:
                    return f"Math Error: {result['error']}"
            else:
                result = self.math_solver.evaluate_expression(prompt)
                if result["success"]:
                    return f"Result: {result['result']}"
                else:
                    return f"Math Error: {result['error']}"
        else:
            return self.generate_with_llm(prompt)


In [4]:
wrapper = LLMMathWrapper(model, tokenizer, solver)
print(wrapper.run("Explain how photosynthesis works."))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Both `max_new_tokens` (=2048) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Explain how photosynthesis works. Photosynthesis is the process by which plants, algae, and some bacteria convert light energy into chemical energy in the form of glucose. The process occurs in the chloroplasts of plant cells, where light energy is absorbed by the chlorophyll molecules in the thylakoid membranes. The absorbed light energy is used to split water molecules into hydrogen and oxygen, which are released into the atmosphere as oxygen. The remaining hydrogen is used to produce glucose, which is the primary source of energy for the plant. The process of photosynthesis is a complex and energy-intensive process that requires a lot of energy to produce glucose, which is why plants are often referred to as the "sun" of the ecosystem.


In [5]:
print(wrapper.run("If a bottle costs $3 and you buy 4, how much total?"))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Both `max_new_tokens` (=2048) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


If a bottle costs $3 and you buy 4, how much total? To determine the total cost of buying 4 bottles of soda, we need to know the cost of one bottle. However, since the problem does not specify the cost of one bottle, we will assume that the cost of one bottle is $3. Here is the step-by-step reasoning:

1. Identify the cost of one bottle of soda.
   - The cost of one bottle is $3.

2. Determine the number of bottles being bought.
   - You are buying 4 bottles.

3. Calculate the total cost by multiplying the cost of one bottle by the number of bottles.
   - Total cost = Cost of one bottle × Number of bottles
   - Total cost = $3 × 4
   - Total cost = $12

Therefore, the total cost of buying 4 bottles of soda is \boxed{12}.


In [6]:
print(wrapper.run("What is the capital of France"))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Both `max_new_tokens` (=2048) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


What is the capital of France?
The capital of France is Paris.


In [7]:
print(wrapper.run("sqrt(x) + sqrt(x + 1) = 2"))

Symbolic: [9/16]
Numeric: [0.562500000000000]


In [8]:
print(wrapper.run("(x - sqrt(3))*(x + sqrt(3)) = 0"))

Symbolic: [-sqrt(3), sqrt(3)]
Numeric: [-1.73205080756888, 1.73205080756888]


In [9]:
print(wrapper.run("x^2 - 4 = 0"))

Symbolic: [-2, 2]
Numeric: [-2.00000000000000, 2.00000000000000]


In [10]:
def reward_func(prompts, completions, completion_ids=None, **kwargs):
    """
    Compute reward for each (prompt, completion) pair.
    Returns a list of floats of the same length as completions.
    """
    rewards = []
    math_solver = MathSolver()

    for prompt, output in zip(prompts, completions):
        # Basic check if it's a math prompt
        is_math = '=' in prompt or any(op in prompt for op in ['+', '-', '*', '/', '^'])

        if is_math:
            try:
                if '=' in prompt:
                    math_result = math_solver.solve_equation(prompt)
                    if math_result["success"]:
                        # If numeric answer is in output, reward it
                        correct_answer = str(math_result["numeric"][0])
                        if correct_answer in output:
                            rewards.append(1.0)
                        else:
                            rewards.append(-1.0)
                    else:
                        rewards.append(-0.5)
                else:
                    eval_result = math_solver.evaluate_expression(prompt)
                    if eval_result["success"] and str(eval_result["result"]) in output:
                        rewards.append(1.0)
                    else:
                        rewards.append(-1.0)
            except Exception:
                rewards.append(-1.0)
        else:
            # For non-math prompts, reward any decent length response
            rewards.append(0.5 if len(output.strip()) > 10 else -0.5)

    return rewards


In [11]:
train_dataset = [
    {"prompt": "Solve x^2 - 4 = 0"},
    {"prompt": "Tell me a joke"},
]


In [12]:
!pip install trl

In [13]:
from trl import GRPOConfig, GRPOTrainer

USE_GPU = False

config = GRPOConfig(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    num_generations=4,
    num_train_epochs=1,
    learning_rate=5e-6,
    logging_steps=2,
    no_cuda=not USE_GPU,
    report_to = "none"
)

# Instantiate GRPOTrainer
grpo_trainer = GRPOTrainer(
    model=model,
    args=config,
    reward_funcs=reward_func,
    train_dataset=train_dataset
)

/usr/local/lib/python3.12/dist-packages/transformers/training_args.py:1619: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(


In [14]:
test_set = [
    {"prompt": "x^2 - 4 = 0"},
    {"prompt": "2 + 2 * 3"},
    {"prompt": "Tell me a joke"},
    {"prompt": "Explain how photosynthesis works."}
]


In [15]:
def evaluate_model(model, tokenizer, test_set):
    model.eval()

    wrapper = LLMMathWrapper(model, tokenizer, MathSolver())
    prompts = [item["prompt"] for item in test_set]

    completions = []
    for prompt in prompts:
        try:
            completions.append(wrapper.run(prompt))
        except Exception as e:
            completions.append(f"[Error in generation: {e}]")

    rewards = reward_func(prompts, completions)

    return list(zip(prompts, completions, rewards))


In [16]:
def compute_accuracy(results):
    correct = sum(1 for _, _, reward in results if reward > 0)
    total = len(results)
    return correct / total if total > 0 else 0.0


In [17]:
# === RUNNING EVALUATION BEFORE TRAINING ===
print("Evaluating model BEFORE GRPO training...")
pre_results = evaluate_model(model, tokenizer, test_set.copy())
pre_accuracy = compute_accuracy(pre_results)

# === TRAINING MODEL ON TRAIN DATASET ===
print("\nStarting GRPO training...")
grpo_trainer.train()
print("Training completed.\n")

# === RUNNING EVALUATION AFTER TRAINING ===
print("Evaluating model AFTER GRPO training...")
post_results = evaluate_model(model, tokenizer, test_set.copy())
post_accuracy = compute_accuracy(post_results)

# === PRINT RESULTS ===
print("\n=== ACCURACY REPORT ===")
print(f"Before GRPO Accuracy: {pre_accuracy:.2%}")
print(f"After  GRPO Accuracy: {post_accuracy:.2%}")

# === Print detailed outputs ===
for prompt, output, reward in post_results:
    print("="*30)
    print(f"Prompt: {prompt}")
    print(f"Output: {output}")
    print(f"Reward: {reward}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Both `max_new_tokens` (=2048) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Evaluating model BEFORE GRPO training...


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Both `max_new_tokens` (=2048) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.



Starting GRPO training...


/usr/local/lib/python3.12/dist-packages/transformers/trainer.py:3964: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  ctx_manager = torch.cpu.amp.autocast(cache_enabled=cache_enabled, dtype=self.amp_dtype)


Step,Training Loss


Both `max_new_tokens` (=2048) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Training completed.

Evaluating model AFTER GRPO training...


Both `max_new_tokens` (=2048) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



=== ACCURACY REPORT ===
Before GRPO Accuracy: 100.00%
After  GRPO Accuracy: 100.00%
Prompt: x^2 - 4 = 0
Output: Symbolic: [-2, 2]
Numeric: [-2.00000000000000, 2.00000000000000]
Reward: 1.0
Prompt: 2 + 2 * 3
Output: Result: 8.00000000000000
Reward: 1.0
Prompt: Tell me a joke
Output: Tell me a joke about a cat.

Why did the cat go to the vet?

Because it was feeling a bit "catatonic"!
Reward: 0.5
Prompt: Explain how photosynthesis works.
Output: Explain how photosynthesis works. Photosynthesis is the process by which plants, algae, and some bacteria convert light energy into chemical energy in the form of glucose. The process occurs in the chloroplasts of plant cells, where light energy is absorbed by the chlorophyll molecules in the thylakoid membranes. The absorbed light energy is used to split water molecules into hydrogen and oxygen, which are released into the atmosphere as oxygen. The remaining hydrogen is used to produce glucose, which is the main source of energy for the plant. 